# Ordinary Differential Equations

In the following, we will work with a few numerical methods to try to solve and characterize ODEs. Since we do not want to solve ODEs explicitly, we first need an algorithm that can tackle this task. 

By far the most simple method is **Euler's method**.


## Euler's method 
For this method, we start by remembering the definition of the derivative

$$f'(t_0) = \lim_{dt\to 0} \frac{f(t_0+dt)-f(t_0)}{dt}$$

So, approximately (without the limit), the derivative of a function at a point t0 is:

$$f'(t_0) \approx \frac{f(t_0+dt)-f(t_0)}{dt}$$

In general, a first order ODE is:

$$\frac{df(t)}{dt} = f'(t) = g(f(t))$$

So, actually, in an ODE we know that the derivative of a function is equal to some other (complex) function of that function $g(f(t))$. Combining the two equations, we get for a specific point $t_0$:

$$g(f(t_0)) = \frac{f(t_0+dt)-f(t_0)}{dt}$$


And, solving for $f(t_0+dt)$, we get

$$f(t_0+dt) = g(f(t_0))*dt + f(t_0)$$

This is an iterative algorithm that allows us to move the solution to the equation a little step $dt$ forward, given that we can evaluate $g(f(t_0))$ and $f(t_0)$. 

We know $g(f(t_0))$, but we do NOT know $f(t_0)$ in general. Therefore, in order to solve this, we need to start at a certain point, which is determined by the initial conditions (also sometimes called boundary conditions). This is necessary for solving ANY ODE!

So, for a certain point $t_0$, we need to know the value of the function at that point

$$f(t_0)=x$$


## Exponential function

Let's do a simple example first. 

We would like to integrate the standard equation that gives us the exponential function.

So, we assume:

$$\frac{df}{dt} = f(t)$$

That is, the derivative of the function $f(t)$ is the function itself. 

In order to start some sort of numerical integration (and to resolve the unknown constant uniquely), we need of course to know some starting point. Let's say, we know that:

$f(0)=a$ and $a=1$

In the following code, try to implement Euler's method based on the equations given above.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# define the equation properly
# parameter a - this is simply a constant
# a = 

# now define the actual ODE with df/dt = v = ?
# this needs to be a function that receives a t-value
# and returns v, of course
# v = 

# step size for Euler integration
dt = 0.1

# total number of integration steps
totNumSteps = 100

# here let's define two arrays: one called ft 
# and another called t. ft contains the solution
# of the ODE and t contains the t-values at which
# we evaluate the solution
# 1) think about how many points we need in each array
# 2) make two arrays ft and t with np.zeros
#
# ft=np.zeros
# t=np.zeros

# since Euler's method is iterative, we need starting
# values for both ft and t
# think about what these are and put them in:
# 
# ft[0]=
# t[0]=

# good, now here we actually do the numerical integration
# so, we need a for-loop that goes through all the steps:
#
# for i in np.arange(<whereDoWeStart?>,<whereDoWeGo?>):
    #
    # in each step of the loop, we now integrate the
    # ODE according to Euler's method. 
    # For this, remember the definition of the derivative
    #      f'(t0) = (f(t0+dt)-f(t0))/dt
    # we do know:
    #      f(t0)     that's our starting point ft[0]
    #      dt        that's our step size
    #      f'(t0)    that's u, our function that we defined
    # we WANT to know:
    #      f(t0+dt)  that's our next point ft[<nextindex>]
    # armed with this, you should be able to solve and
    # update the next values for ft and t
    #
    # ft[i]=
    # t[i]=
    

# now let's plot this and compare to the true solution
plt.plot(t,ft,t,np.exp(t))
plt.show()



### Using `odeint` from `scipy`

Now, let's use the professional toolkit! `scipy` has a function called `odeint` that solves first-order ODEs for you using much more sophisticated algorithms (such as Runge-Kutta methods, etc.). We set this up in the following way:

In [ ]:
from scipy import integrate
from numpy import linspace,exp
import matplotlib.pylab as plt

def df_dt(y,t=0):
    # y is the current function value 
    # t is the current time point - we usually don't need to explicitly work 
    # with this, so we set this to 0 as a default argument
    
    # Change the following code to return the correct function value for the
    # ODE we want to investigate
    return(0)

# total time we want to get our data for
t = linspace(0, 10,  100)            

# initial function value
f0 = 1                    

# let's use the integration method of scipy
ft = integrate.odeint(df_dt, f0, t)

plt.plot(t,ft)
plt.plot(t,exp(t),'r--')
plt.show()

## Extending the simple example

Now, let's upgrade our population function to something nicer. Let's say, we know that food is limited and that there is a maximum population that our environment can support.

So, the ODE of growth we had before:

$$\frac{df}{dt} = f(t)$$

does not hold anymore. We need to change it to:

$$\frac{df}{dt} = f(t)*\left( 1-\frac{f(t)}{K} \right)$$

where $K$ is carrying capacity.

In [ ]:
from scipy import integrate
from numpy import linspace,exp
import matplotlib.pylab as plt

def df_dt(y,t,K):
    # y is the current function value 
    # t is the current time point - we usually don't need to explicitly work 
    # with this
    # K is an additional parameter that indicates the carrying capacity of our model
    
    # Change the following code to return the correct function value for the
    # ODE we want to investigate
    return(0)

# total time we want to get our data for
t = linspace(0, 20,  100)            

# initial function value
f0 = 1             

# carrying capacity
K = 1000

# let's use the integration method of scipy
# note that "K" is passed as a tuple to scipy
ft = integrate.odeint(df_dt, f0, t, args=(K,))

plt.plot(t,ft)
plt.plot(t,exp(t),'r--')
plt.plot([0,20],[1000,1000],'k--')
plt.ylim(0,1100)
plt.show()

## Rabbits and wolves
We will have a look at the Lokta-Volterra model, also known as the predator-prey equations, which are a pair of first order, non-linear, differential equations frequently used to describe the dynamics of biological systems in which two species interact, one a predator and one its prey. They were proposed independently by Alfred J. Lotka in 1925 and Vito Volterra in 1926:

$$
\begin{eqnarray}
\frac{dR}{dt} &=&  k*R &-&   a*R*W\\
\frac{dW}{dt} &=& -r*W &+&   b*R*W\\
\end{eqnarray}
$$

with the following notations:

* $R$: number of preys (for example, rabbits)
* $W$: number of predators (for example, wolves)  
* $k, a, r, b$ as constant parameters defining the behavior of the population:
    * $k$ is the natural growing rate of rabbits, when there's no wolf
    * $a$ is the natural dying rate of rabbits, due to predation
    * $r$ is the natural dying rate of a wolf, when there's no rabbit
    * $b$ is the factor describing how many caught rabbits can support a new wolf

It is convenient to combine the two populations in one vector as follows:
$$\vec{X}=\left(
\begin{array}{c}
R\\
W\\
\end{array}
\right)$$



In [ ]:
# This example describes how to integrate ODEs with scipy.integrate module, and how
# to use the matplotlib module to plot trajectories, direction fields and other
# useful information.
# 
# This code will combine the two equations in one array:
# X=[R, W] to describe the state of both populations.
#
# Definition of the equations:
# 
from numpy import *
import matplotlib.pylab as plt

# Definition of parameters 
k = 0.08
a = 0.001
r = 0.02
b = 0.00002

def dX_dt(X, t=0):
    """ Return the growth rate of fox and rabbit populations. """
    return array([ k*X[0] -   a*X[0]*X[1] ,  
                  -r*X[1] +   b*X[0]*X[1] ])


### Population equilibrium  
Before using solving this system, we will have a closer look at the equilibrium. 

Equilibrium occurs when the growth rate $\frac{d\vec{X}}{dt}$ is equal to 0. This gives two fixed points:

In [ ]:
# first fixed point (everything is 0)
X_f0 = array([     0. ,  0.])
# second fixed point (when dR/dt and dW/dt =0)
X_f1 = array([ r/b, k/a])
print('Second fixed point with',X_f1[0],'rabbits and',X_f1[1],'wolves')
# both points should of course have 0 growth rate,
# so let's check this in python:
all(dX_dt(X_f0) == zeros(2) ) and all(dX_dt(X_f1) == zeros(2)) # => True 


### Stability of the fixed points
Near these two points, the system can be linearized:
$$\frac{d\vec{X}}{dt} \approx \mathbf{A}*\vec{X}$$
where $\mathbf{A}$ is the Jacobian matrix evaluated at the corresponding point.
$$A=\left(
\begin{array}{cc}
\frac{dR/dt}{dR}&\frac{dR/dt}{dW}\\
\frac{dW/dt}{dR}&\frac{dW/dt}{dW}\\
\end{array}
\right)$$

This is an extension of the simple Taylor expansion.

In [ ]:
# 
# Let's first define the Jacobian matrix:
# 
def d2X_dt2(X, t=0):
    """ Return the Jacobian matrix evaluated in X. """
    return array([[k -a*X[1],   -a*X[0]     ],
                  [b*X[1] ,   -r +b*X[0]] ])  
# 
# So, near X_f0, which represents the extinction of 
# both species, we have:
A_f0 = d2X_dt2(X_f0)
print('changes around first fixpoint:\n',A_f0)
# Near X_f0, the number of rabbits increases 
# and the population of wolves decreases.
# The origin is therefore a saddle point!
# 
# Near X_f1, we have:
A_f1 = d2X_dt2(X_f1) 
print('changes around second fixpoint:\n',A_f1)

# The eigenvalues of the Jacobian are +/- sqrt(k*r)*j
lambda1, lambda2 = linalg.eigvals(A_f1) 
print('\ncalculated eigenvalues:\n',lambda1,lambda2,"\ncompare with theoretical",sqrt(k*r))

# They are imaginary numbers, so that the wolf and 
# rabbit populations are periodic and their approximate 
# period will be given by:
T_f1 = 2*pi/abs(lambda1)
print('\n\napproximate period of wolf/rabbit population:\n',T_f1)

### Integrating the ODE using scipy.integate 
Now we will use the scipy.integrate module to integrate the ODEs. In addition, we will be using the solution to try and find the actual period.

In [ ]:

from scipy import integrate, signal

# total time we want to get our data for
t = linspace(0, 1500,  1000)            

# initial population of rabbits and wolves
X0 = array([1000, 40])                    

# let's use the integration method of scipy
X = integrate.odeint(dX_dt, X0, t)

# We can now use Matplotlib to plot the 
# evolution of both populations:

rabbits, wolves = X.T

# find the peaks in the signal
rp=signal.find_peaks_cwt(rabbits,[10,1000])
wp=signal.find_peaks_cwt(wolves,[10,1000])

# since we have a periodic signal, the period is 
# (hopefully) the distance between the first two peaks
rperiod=t[rp[1]]-t[rp[0]]
wperiod=t[wp[1]]-t[wp[0]]

f1,ax1 = plt.subplots()
ax1.plot(t, rabbits, 'r-', label='Rabbits')
ax1.grid()
ax1.set_xlabel('time')
ax1.set_ylabel('rabbit population',color='r')
ax1.tick_params('y', colors='r')
ax2=ax1.twinx()
ax2.plot(t, wolves  , 'b-', label='wolves')
ax2.set_ylabel('wolf population',color='b')
ax2.tick_params('y', colors='b')
plt.title('Rabbit and wolf populations\nRabbit period %i, Wolf period %i'%(rperiod,wperiod))
plt.show()
f1.savefig('rabbits_and_wolves_1.png')

### Plotting direction fields and trajectories in the phase plane
We will plot some trajectories in a phase plane for different starting points between X_f0 and X_f1.
 
We will use matplotlib's colormap to define colors for the trajectories. These colormaps are very useful to make nice plots. Have a look at 

http://www.scipy.org/Cookbook/Matplotlib/Show_colormaps 

if you want more information.

In [ ]:
# we wil evaluate points between the two
# equilibrium points X_f0 and X_f1
values  = linspace(0.3, 0.9, 5)
# let's get some fancy colors for each trajectory
vcolors = plt.cm.autumn_r(linspace(0.3, 1., len(values)))  
f2 = plt.figure()

# plot trajectories
for v, col in zip(values, vcolors): 
    # starting point
    X0 = v * X_f1
    # integrate the ODE for the times and starting points
    X = integrate.odeint( dX_dt, X0, t)
    # plot the trajectory with varying linewidth and color
    plt.plot( X[:,0], X[:,1], lw=3.5*v, color=col, label='X0=(%.f, %.f)' % ( X0[0], X0[1]) )
    plt.plot( X0[0], X0[1], 'xk')
    
# define a grid and compute direction at each point
# get axis limits
ymax = plt.ylim(ymin=0)[1]                        
xmax = plt.xlim(xmin=0)[1] 
nb_points   = 30                      

x = linspace(0, xmax, nb_points)
y = linspace(0, ymax, nb_points)
# create a grid with the axis limits
X1 , Y1  = meshgrid(x, y)    
# compute growth rate on the grid!
DX1, DY1 = dX_dt([X1, Y1])    
# Norm of the growth rate 
M = sqrt(DX1**2+DY1**2)     
# Avoid zero division errors - for example, 
# in case we hit the stability point!
M[ M == 0] = 1.      
# Normalize each arrow by the growth rate's norm
DX1 /= M                                        
DY1 /= M                                  

#-------------------------------------------------------
# Draw direction fields, using matplotlib's quiver-function
# Arrows are plotted in the same size, but colors are used 
# to give information about the growth speed
plt.title('Trajectories and direction fields')
Q = plt.quiver(X1, Y1, DX1, DY1, M, pivot='mid', cmap=plt.cm.jet)
plt.xlabel('Number of rabbits')
plt.ylabel('Number of wolves')
plt.legend()
plt.grid()
plt.xlim(0, xmax)
plt.ylim(0, ymax)
plt.show()
f2.savefig('rabbits_and_wolves_2.png')
# 
# 
# We can see on this graph that an intervention for
# both wolf or rabbit populations can have non-intuitive 
# effects. If, in order to decrease the number of rabbits,
# we introduce wolves, for example, this can lead to an 
# **increase** of rabbits in the long run, if that 
# intervention happens at a bad moment.
